<a href="https://colab.research.google.com/github/Justy-11/deeplearning_fundamentals/blob/main/07_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install keras-tuner --upgrade

In [3]:
import keras_tuner
from tensorflow import keras

Using TensorFlow backend


In [4]:
df = pd.read_csv('/content/diabetes.csv')

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
# correlation between Outcome and others
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [7]:
# [:,:-1]: This part of the code specifies the row and column selection. Here, : is used for selecting all rows, and :-1 is used for selecting all columns except the last one.
# In other words, it selects all rows and all columns up to, but not including, the last column in the DataFrame.
X = df.iloc[:,:-1].values
y = df.iloc[:, -1].values

In [8]:
X.shape, y.shape

((768, 8), (768,))

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
# scale from -1 to 1
X = scaler.fit_transform(X)

In [11]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42)

In [13]:
X_train, y_train

(array([[ 1.82781311e+00, -1.43636781e+00,  6.66618252e-01, ...,
          1.65936998e-01, -6.30831221e-01,  4.04942367e-01],
        [ 9.36913723e-01,  5.04055196e-01,  1.08020025e+00, ...,
          9.41978774e-04, -2.44256030e-01,  4.90030120e-01],
        [-2.50952128e-01,  1.16129525e+00, -2.63941247e-01, ...,
         -1.00593418e-01, -5.34187424e-01, -7.86286177e-01],
        ...,
        [ 1.82781311e+00, -6.22642036e-01,  8.73409251e-01, ...,
          1.72704372e+00,  2.00573238e+00,  4.04942367e-01],
        [-1.14185152e+00,  6.29243777e-01, -3.57259724e+00, ...,
          1.32090213e+00, -8.05998104e-01, -3.60847411e-01],
        [-1.14185152e+00,  1.28489452e-01,  1.39038675e+00, ...,
         -1.20479085e+00, -6.33851340e-01, -1.04154944e+00]]),
 array([0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       

In [14]:
import keras_tuner as kt

# Hp - no. of units in layers,
# Hp - activation functions
# Hp - optimizers

In [15]:
# hp argument to define the hyperparameters during model creation.
def build_model(hp):
  model = keras.Sequential()

  # model.add(keras.layers.Dense(
  #     hp.Choice('units', [8, 16, 32]),
  #     activation='relu'))

  model.add(keras.layers.Dense(
      units=hp.Choice('units', [8, 16, 32]),
      activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid']),
  ))

  model.add(keras.layers.Dense(
      units=hp.Choice('units2', [16, 32, 64]),
      activation=hp.Choice('activation2', ['relu', 'tanh', 'sigmoid']),
  ))

  model.add(keras.layers.Dense(1, activation='relu'))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

  return model

In [16]:
# objective='val_accuracy': This specifies the objective to optimize during the hyperparameter search. In this case, the objective is to maximize the validation accuracy of the model.
# The hyperparameter tuning process will try different hyperparameter combinations to find the configuration that produces the best validation accuracy.
# max_trials=15: This sets the maximum number of trials (i.e., hyperparameter combinations) to be tested during the random search. In this example, the hyperparameter search will
# run a maximum of 15 trials, each with a different set of hyperparameters.
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15)

In [17]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 15 Complete [00h 00m 02s]
val_accuracy: 0.6585366129875183

Best val_accuracy So Far: 0.7398374080657959
Total elapsed time: 00h 00m 46s


In [18]:
best_model = tuner.get_best_models()[0]

In [19]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 10 summary
Hyperparameters:
units: 8
activation: sigmoid
units2: 16
activation2: relu
optimizer: adam
Score: 0.7398374080657959

Trial 01 summary
Hyperparameters:
units: 16
activation: relu
units2: 16
activation2: tanh
optimizer: adam
Score: 0.7154471278190613

Trial 00 summary
Hyperparameters:
units: 16
activation: sigmoid
units2: 16
activation2: relu
optimizer: sgd
Score: 0.6715447306632996

Trial 13 summary
Hyperparameters:
units: 8
activation: relu
units2: 32
activation2: relu
optimizer: adadelta
Score: 0.6650406718254089

Trial 07 summary
Hyperparameters:
units: 8
activation: tanh
units2: 16
activation2: relu
optimizer: adam
Score: 0.6634146571159363

Trial 05 summary
Hyperparameters:
units: 32
activation: relu
units2: 32
activation2: sigmoid
optimizer: sgd
Score: 0.660162627696991

Trial 02 summary
Hyperparameters:
units: 16
activation: sigmoid
units2: 32
ac

In [20]:
# !rm -r /content/untitled_project

In [30]:
def build_model(hp):
  model = keras.Sequential()

  units = hp.Int('units', min_value=8, max_value=128, step=8)

  model.add(keras.layers.Dense(
      units=units,
      activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid']),
  ))

  model.add(keras.layers.Dense(1, activation='relu'))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

  return model

In [31]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=15,
    directory='dir',
    project_name='hp_2')

In [32]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 15 Complete [00h 00m 02s]
val_accuracy: 0.7284553050994873

Best val_accuracy So Far: 0.7560975551605225
Total elapsed time: 00h 00m 34s


In [33]:
best_model = tuner.get_best_models()[0]

In [34]:
tuner.results_summary()

Results summary
Results in dir/hp_2
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 13 summary
Hyperparameters:
units: 8
activation: tanh
optimizer: sgd
Score: 0.7560975551605225

Trial 04 summary
Hyperparameters:
units: 64
activation: tanh
optimizer: rmsprop
Score: 0.7544715404510498

Trial 11 summary
Hyperparameters:
units: 104
activation: tanh
optimizer: sgd
Score: 0.7430894374847412

Trial 14 summary
Hyperparameters:
units: 96
activation: tanh
optimizer: rmsprop
Score: 0.7284553050994873

Trial 00 summary
Hyperparameters:
units: 56
activation: tanh
optimizer: sgd
Score: 0.707317054271698

Trial 01 summary
Hyperparameters:
units: 104
activation: tanh
optimizer: rmsprop
Score: 0.6926829218864441

Trial 06 summary
Hyperparameters:
units: 32
activation: relu
optimizer: sgd
Score: 0.6796748042106628

Trial 02 summary
Hyperparameters:
units: 24
activation: tanh
optimizer: sgd
Score: 0.6780487895011902

Trial 07 summary
Hyperparameters:
units: 24
activation: 

In [35]:
tuner.get_best_hyperparameters()[0].values

{'units': 8, 'activation': 'tanh', 'optimizer': 'sgd'}

In [38]:
model = tuner.get_best_models(num_models=1)[0]

In [39]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6)

Epoch 7/100
5/5 [==============================] - 1s 10ms/step - loss: 1.4971 - accuracy: 0.7320
Epoch 8/100
5/5 [==============================] - 0s 6ms/step - loss: 1.4767 - accuracy: 0.7386
Epoch 9/100
5/5 [==============================] - 0s 6ms/step - loss: 1.1526 - accuracy: 0.7320
Epoch 10/100
5/5 [==============================] - 0s 6ms/step - loss: 1.0102 - accuracy: 0.7320
Epoch 11/100
5/5 [==============================] - 0s 6ms/step - loss: 0.9998 - accuracy: 0.7190
Epoch 12/100
5/5 [==============================] - 0s 5ms/step - loss: 0.9948 - accuracy: 0.7190
Epoch 13/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9917 - accuracy: 0.7190
Epoch 14/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9888 - accuracy: 0.7124
Epoch 15/100
5/5 [==============================] - 0s 4ms/step - loss: 0.9898 - accuracy: 0.7124
Epoch 16/100
5/5 [==============================] - 0s 5ms/step - loss: 0.9880 - accuracy: 0.7124
Epoch 17/100
5/5 [====

# Hp - no of layers

In [40]:
def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.Dense(72, activation='relu'))

  for i in range(hp.Int('no of layers', min_value=1, max_value=10)):
    model.add(keras.layers.Dense(72, activation='relu'))

  model.add(keras.layers.Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')

  return model

In [41]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='dir',
    project_name='hp_3')

In [42]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.7821138501167297

Best val_accuracy So Far: 0.7821138501167297
Total elapsed time: 00h 00m 15s


In [44]:
best_model = tuner.get_best_models()[0]

In [45]:
tuner.get_best_hyperparameters()[0].values

{'no of layers': 2}

In [46]:
tuner.results_summary()

Results summary
Results in dir/hp_3
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
no of layers: 2
Score: 0.7821138501167297

Trial 0 summary
Hyperparameters:
no of layers: 1
Score: 0.7528455257415771

Trial 1 summary
Hyperparameters:
no of layers: 8
Score: 0.7528455257415771


In [47]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train, y_train, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
5/5 [==============================] - 1s 73ms/step - loss: 0.5022 - accuracy: 0.7908 - val_loss: 0.4957 - val_accuracy: 0.7772
Epoch 8/100
5/5 [==============================] - 0s 27ms/step - loss: 0.4750 - accuracy: 0.7908 - val_loss: 0.4841 - val_accuracy: 0.7756
Epoch 9/100
5/5 [==============================] - 0s 20ms/step - loss: 0.4504 - accuracy: 0.7974 - val_loss: 0.4746 - val_accuracy: 0.7772
Epoch 10/100
5/5 [==============================] - 0s 25ms/step - loss: 0.4334 - accuracy: 0.7843 - val_loss: 0.4735 - val_accuracy: 0.7756
Epoch 11/100
5/5 [==============================] - 0s 20ms/step - loss: 0.4189 - accuracy: 0.7843 - val_loss: 0.4741 - val_accuracy: 0.7772
Epoch 12/100
5/5 [==============================] - 0s 19ms/step - loss: 0.4049 - accuracy: 0.8039 - val_loss: 0.4796 - val_accuracy: 0.7724
Epoch 13/100
5/5 [==============================] - 0s 20ms/step - loss: 0.3929 - accuracy: 0.8039 - val_loss: 0.4856 - val_accuracy: 0.7691
Epoch 14/100
5/5

In [49]:
def build_model(hp):
  model = keras.Sequential()

  counter = 0

  for i in range(hp.Int('no of layers', min_value=1, max_value=10)):
    if counter == 0:
      model.add(keras.layers.Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                                 activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid']),
                                 input_dim=8))
    else:
      model.add(keras.layers.Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                                 activation=hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])))
    counter += 1

  model.add(keras.layers.Dense(1, activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')

  return model

In [50]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=3,
    directory='dir',
    project_name='hp_4')

In [51]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6585366129875183

Best val_accuracy So Far: 0.6585366129875183
Total elapsed time: 00h 00m 12s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'no of layers': 10,
 'units0': 96,
 'activation': 'relu',
 'optimizer': 'adadelta',
 'units1': 8,
 'units2': 8,
 'units3': 8,
 'units4': 8,
 'units5': 8,
 'units6': 8,
 'units7': 8,
 'units8': 8,
 'units9': 8}

In [53]:
tuner.results_summary()

Results summary
Results in dir/hp_4
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
no of layers: 10
units0: 96
activation: relu
optimizer: adadelta
units1: 8
units2: 8
units3: 8
units4: 8
units5: 8
units6: 8
units7: 8
units8: 8
units9: 8
Score: 0.6585366129875183

Trial 2 summary
Hyperparameters:
no of layers: 3
units0: 72
activation: sigmoid
optimizer: rmsprop
units1: 104
units2: 64
units3: 104
units4: 120
units5: 96
units6: 88
units7: 96
units8: 72
units9: 72
Score: 0.6585366129875183

Trial 1 summary
Hyperparameters:
no of layers: 8
units0: 128
activation: sigmoid
optimizer: sgd
units1: 96
units2: 56
units3: 64
units4: 8
units5: 120
units6: 64
units7: 8
units8: 48
units9: 112
Score: 0.3414634168148041


In [55]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train, y_train, batch_size=32, epochs=200, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/200
5/5 [==============================] - 2s 87ms/step - loss: 0.6920 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 8/200
5/5 [==============================] - 0s 21ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 9/200
5/5 [==============================] - 0s 20ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 10/200
5/5 [==============================] - 0s 19ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 11/200
5/5 [==============================] - 0s 19ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 12/200
5/5 [==============================] - 0s 19ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 13/200
5/5 [==============================] - 0s 26ms/step - loss: 0.6919 - accuracy: 0.6209 - val_loss: 0.6914 - val_accuracy: 0.6585
Epoch 14/200
5/5